In [1]:
import netCDF4 as nc
from geoarray import GeoArray
from spectral.io import envi
import matplotlib.pyplot as plt
import numpy as np
from isofit.core.common import resample_spectrum
import os
import isofit
from isofit.core.sunposition import sunpos
import matplotlib
import tkinter
matplotlib.use('TkAgg')

C:\Users\vpatro\AppData\Local\anaconda3\envs\isofit_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# relevant wavelenghts for water vapor: 900-1050 (begin with first feature)
# second: 1100-1300

# trainining split: speak to Jake and Michael

# ideas from Niklas: start with linear regression, lasso regression, maybe perceptron, polynomial regression
# take TOA reflectance and do PCA -- project those onto water vapor analysis space

# image segmenetation: principle component space

In [3]:
rad = '../west_africa_scene/EMIT_L1B_RAD_001_20230321T150833_2308010_003.nc'
ds_rad = nc.Dataset(rad)
mask = '../west_africa_scene/EMIT_L2A_MASK_001_20230321T150833_2308010_003.nc'
ds_mask = nc.Dataset(mask)

In [4]:
#ds_rad

In [5]:
# rho = (((rdn * np.pi) / (irr.T)).T / np.cos(zen)).T

In [6]:
#west_africa_coast = GeoArray(r'C:\\Users\\vpatro\\Desktop\\west_africa_scene\\EMIT_L1B_RAD_001_20230321T150833_2308010_003_radiance')

In [7]:
refl_header = envi.open('../west_africa_scene/EMIT_L2A_RFL_001_20230321T150833_2308010_003_reflectance.hdr')
print(type(refl_header))
#print(ds.metadata)
dat_refl = refl_header.open_memmap(interleave='bip')

# plot an RGB from reflectance data
plt.imshow(dat_refl[:,:,np.array([30,21,12])])

C:\Users\vpatro\AppData\Local\anaconda3\envs\isofit_env\lib\site-packages\spectral\io\envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)


<class 'spectral.io.bilfile.BilFile'>


In [8]:
# using spectral library
mask_header = envi.open('../west_africa_scene/EMIT_L2A_MASK_001_20230321T150833_2308010_003_mask.hdr')
dat_mask = mask_header.open_memmap(interleave='bip')
dat_mask.shape

(1888, 1912, 8)

In [ ]:
plt.imshow(dat_mask[:,:,6])
plt.colorbar()
plt.title("Atmospheric Water Vapor Levels (g/cm^2)")
plt.show()

In [12]:
plt.hist(dat_mask[:,:,6].flatten())
plt.show()

In [11]:
#mask_header.metadata

In [13]:
rad_header = envi.open('../west_africa_scene/EMIT_L1B_RAD_001_20230321T150833_2308010_003_radiance.hdr')
print(type(rad_header))
#print(ds.metadata)
dat_rad = rad_header.open_memmap(interleave='bip')

<class 'spectral.io.bilfile.BilFile'>


In [17]:
plt.plot(dat_rad[750,750,:])
plt.title('Radiance Spectra')
plt.xlabel('Band Number')

Text(0.5, 0, 'Band Number')

In [90]:
# rho = (((rdn * np.pi) / (irr.T)).T / np.cos(zen)).T

# zen should be one number (set Rad = True)
# irr should be an array with dimension equal to rdn

In [106]:
type(ds_rad)
ds_rad.__dict__

{'ncei_template_version': 'NCEI_NetCDF_Swath_Template_v2.0',
 'summary': "The Earth Surface Mineral Dust Source Investigation (EMIT) is an Earth Ventures-Instrument (EVI-4) Mission that maps the surface mineralogy of arid dust source regions via imaging spectroscopy in the visible and short-wave infrared (VSWIR). Installed on the International Space Station (ISS), the EMIT instrument is a Dyson imaging spectrometer that uses contiguous spectroscopic measurements from 410 to 2450 nm to resolve absoprtion features of iron oxides, clays, sulfates, carbonates, and other dust-forming minerals. During its one-year mission, EMIT will observe the sunlit Earth's dust source regions that occur within +/-52° latitude and produce maps of the source regions that can be used to improve forecasts of the role of mineral dust in the radiative forcing (warming or cooling) of the atmosphere.\\n\\nThis file contains L1B at-sensor calibrated radiances. The radiance calibration occurs in two basic stages: 1

In [107]:
ds_rad.__dict__.keys()

dict_keys(['ncei_template_version', 'summary', 'keywords', 'Conventions', 'sensor', 'instrument', 'platform', 'institution', 'license', 'naming_authority', 'date_created', 'keywords_vocabulary', 'stdname_vocabulary', 'creator_name', 'creator_url', 'project', 'project_url', 'publisher_name', 'publisher_url', 'publisher_email', 'identifier_product_doi_authority', 'flight_line', 'time_coverage_start', 'time_coverage_end', 'software_build_version', 'software_delivery_version', 'product_version', 'history', 'crosstrack_orientation', 'easternmost_longitude', 'northernmost_latitude', 'westernmost_longitude', 'southernmost_latitude', 'spatialResolution', 'spatial_ref', 'geotransform', 'day_night_flag', 'title'])

In [109]:
ds_rad.__dict__['spatialResolution']

0.000542232520256367

In [ ]:
# what is irradiance - understood
# use load_wavelength to get wl and fwhm but need file paths and need to extract out info from netCDF structure


# early thoughts on ML